In [1]:
def multiplier(a, b):
    """ 
    Here we perform one digit multiplication using 13 qubits(1 Ancillary qubit) and 6 bits classical register for measurement 
    for now limitation are, we can only perform multiplication up to number 7('111') so the maximum result we get is 49 however for measurement range is 63('111111').
    """
    # Importing standard Qiskit libraries
    import numpy as np
    from qiskit import QuantumCircuit, execute, ClassicalRegister, QuantumRegister
    from qiskit.quantum_info import Statevector
    from qiskit.providers.aer import QasmSimulator, Aer
    # Getting provider for experiment here we use noise-less qasm simulator
    backend = Aer.get_backend('qasm_simulator')
    ####################################################
    # Creating Quantum Circuit   
    q = QuantumRegister(13)
    c = ClassicalRegister(6)
    circuit = QuantumCircuit(q,c)
    #################################################
    # changing integers into binary strings
    def decimalToBinary(n):
        return bin(n).replace("0b", "")
    p = decimalToBinary(a)
    q = decimalToBinary(b)
    #################################################
    # preparing initial state
    sv = Statevector.from_label(p)
    circuit.initialize(sv, [0,1,2])
    sq = Statevector.from_label(q)
    circuit.initialize(sq, [3,4,5])
    #################################################
    # Importing Multipier and Adder
    from typing import Optional
    from qiskit.circuit import QuantumRegister, AncillaRegister, QuantumCircuit
    from qiskit.circuit.library.arithmetic.adders.cdkm_ripple_carry_adder import Adder
    from qiskit.circuit.library.arithmetic.multipliers.multiplier import Multiplier
    #################################################
    class Adder(Adder):
        def __init__(
            self, num_state_qubits: int, kind: str = "full", name: str = "Adder"
        ) -> None:
            super().__init__(num_state_qubits, name=name)
            if kind == "full":
                qr_c = QuantumRegister(1, name="cin")
                self.add_register(qr_c)
            else:
                qr_c = AncillaRegister(1, name="help")
            qr_a = QuantumRegister(num_state_qubits, name="a")
            qr_b = QuantumRegister(num_state_qubits, name="b")
            self.add_register(qr_a, qr_b)
            if kind in ["full", "half"]:
                qr_z = QuantumRegister(1, name="cout")
                self.add_register(qr_z)
            if kind != "full":
                self.add_register(qr_c)
            # build carry circuit for majority of 3 bits in-place
            # corresponds to MAJ gate in [1]
            qc_maj = QuantumCircuit(3, name="MAJ")
            qc_maj.cx(0, 1)
            qc_maj.cx(0, 2)
            qc_maj.ccx(2, 1, 0)
            maj_gate = qc_maj.to_gate()
            # build circuit for reversing carry operation
            # corresponds to UMA gate in [1]
            qc_uma = QuantumCircuit(3, name="UMA")
            qc_uma.ccx(2, 1, 0)
            qc_uma.cx(0, 2)
            qc_uma.cx(2, 1)
            uma_gate = qc_uma.to_gate()
            circuit = QuantumCircuit(*self.qregs, name=name)
            # build ripple-carry adder circuit
            circuit.append(maj_gate, [qr_a[0], qr_b[0], qr_c[0]])
            for i in range(num_state_qubits - 1):
                circuit.append(maj_gate, [qr_a[i + 1], qr_b[i + 1], qr_a[i]])
            if kind in ["full", "half"]:
                circuit.cx(qr_a[-1], qr_z[0])
            for i in reversed(range(num_state_qubits - 1)):
                circuit.append(uma_gate, [qr_a[i + 1], qr_b[i + 1], qr_a[i]])
            circuit.append(uma_gate, [qr_a[0], qr_b[0], qr_c[0]])
            self.append(circuit.to_gate(), self.qubits)
    #################################################
    class Multiplier(Multiplier):
        def __init__(
            self,
            num_state_qubits: int,
            num_result_qubits: Optional[int] = None,
            adder: Optional[QuantumCircuit] = None,
            name: str = "Multiplier",
        ) -> None:
            super().__init__(num_state_qubits, num_result_qubits, name=name)
            # define the registers
            qr_a = QuantumRegister(num_state_qubits, name="a")
            qr_b = QuantumRegister(num_state_qubits, name="b")
            qr_out = QuantumRegister(self.num_result_qubits, name="out")
            self.add_register(qr_a, qr_b, qr_out)
            # prepare adder as controlled gate
            if adder is None:
                adder = Adder(num_state_qubits, kind="half") 
            # get the number of helper qubits needed
            num_helper_qubits = adder.num_ancillas
            # add helper qubits if required
            if num_helper_qubits > 0:
                # helper ubits/ ancilla qubits
                qr_h = AncillaRegister(num_helper_qubits, name="helper")
                self.add_register(qr_h)
            # build multiplication circuit
            circuit = QuantumCircuit(*self.qregs, name=name)
            for i in range(num_state_qubits):
                excess_qubits = max(0, num_state_qubits + i + 1 - self.num_result_qubits)
                if excess_qubits == 0:
                    num_adder_qubits = num_state_qubits
                    adder_for_current_step = adder
                else:
                    num_adder_qubits = num_state_qubits - excess_qubits + 1
                    adder_for_current_step = Adder(num_adder_qubits, kind="fixed")   
                controlled_adder = adder_for_current_step.to_gate().control(1)
                qr_list = (
                    [qr_a[i]]
                    + qr_b[:num_adder_qubits]
                    + qr_out[i : num_state_qubits + i + 1 - excess_qubits]
                )
                if num_helper_qubits > 0:
                    qr_list.extend(qr_h[:])
                circuit.append(controlled_adder, qr_list)
            self.append(circuit.to_gate(), self.qubits)
    circuit1 = Multiplier(num_state_qubits=3, num_result_qubits=6, name='Multiplier')
    circuit = circuit.compose(circuit1)
    # Measurment
    circuit.measure(6, 0)
    circuit.measure(7, 1)
    circuit.measure(8, 2)
    circuit.measure(9, 3)
    circuit.measure(10,4)
    circuit.measure(11,5)
    # running experiment
    job = execute(circuit, backend, shots=2048)
    result = job.result()
    counts = result.get_counts()
    # converting caounts:dict to list[]
    dict = counts
    list = [(k) for k, v in dict.items()]
    # converting list to bit-string
    def listToString(s):
        # initialize an empty string
        str1 = ""  
        # traverse in the string
        for ele in s:
            str1 += ele   
        # return string
        return str1
    ###########################################   
    s = list
    listToString(s)
    # get Integer from Binary String
    product = int(listToString(s), 2)
    # return product of params integers
    return product

In [2]:
%%time
multiplier(5,6)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


CPU times: user 27.7 s, sys: 2.72 s, total: 30.4 s
Wall time: 4.83 s


30